### Artificial intelligence monthly momemtum strategy

In [1]:
# get category symbols
category = "artificial-intelligence"
import requests
url = "https://api.coingecko.com/api/v3/coins/markets"
params = {
    "vs_currency": "usd",
    "category": category,
    "order": "market_cap_desc",
    "per_page": 300,
    "page": 1
}

response = requests.get(url, params=params)
data = response.json()

# Extract the symbols
defi_symbols = [coin['symbol'].upper() + '/USDT' for coin in data]
print("DeFi Symbols:", defi_symbols)

DeFi Symbols: ['NEAR/USDT', 'ICP/USDT', 'TAO/USDT', 'FET/USDT', 'RENDER/USDT', 'GRT/USDT', 'AIOZ/USDT', 'AKT/USDT', 'GRASS/USDT', 'ARKM/USDT', 'VIRTUAL/USDT', 'LPT/USDT', 'GLM/USDT', 'TRAC/USDT', 'IO/USDT', 'NOS/USDT', 'AGI/USDT', 'AGIX/USDT', 'FLUX/USDT', 'QUBIC/USDT', 'ZIG/USDT', 'SPEC/USDT', 'BASEDAI/USDT', 'OCEAN/USDT', 'IQ/USDT', 'CARV/USDT', 'HEART/USDT', 'OLAS/USDT', 'RLC/USDT', '0X0/USDT', 'NMR/USDT', 'PAAL/USDT', 'TOKEN/USDT', 'ORAI/USDT', 'NMT/USDT', 'POND/USDT', 'CGPT/USDT', 'PHB/USDT', 'PHA/USDT', 'LUNA/USDT', 'AITECH/USDT', 'NEURAL/USDT', 'ALI/USDT', 'GPU/USDT', 'TAI/USDT', 'RSS3/USDT', 'DEAI/USDT', 'CUDOS/USDT', 'ARC/USDT', 'AI/USDT', 'CTXC/USDT', 'CXT/USDT', 'PALM/USDT', 'FORT/USDT', 'OCTA/USDT', 'LMWR/USDT', 'NUM/USDT', 'AGRS/USDT', 'BOTTO/USDT', 'CLORE/USDT', 'LNQ/USDT', 'LAI/USDT', 'ALEPH/USDT', 'VAI/USDT', 'JKL/USDT', 'NAVI/USDT', 'HGPT/USDT', 'TRIAS/USDT', 'COMAI/USDT', 'DCD/USDT', 'GLQ/USDT', 'GTAI/USDT', 'SPECTRE/USDT', 'AIT/USDT', 'NAI/USDT', 'XMW/USDT', 'TIG/USD

In [2]:
# Fetch aviable symbols from Bybit
import ccxt

exchange = ccxt.bybit()
exchange.load_markets()
all_symbols = exchange.symbols

# Filter to include only DeFi symbols available on the exchange
available_defi_symbols = [symbol for symbol in defi_symbols if symbol in all_symbols]
print("Available DeFi Symbols:", available_defi_symbols)

Available DeFi Symbols: ['NEAR/USDT', 'ICP/USDT', 'FET/USDT', 'RENDER/USDT', 'GRT/USDT', 'AIOZ/USDT', 'GRASS/USDT', 'ARKM/USDT', 'IO/USDT', 'AGI/USDT', 'ZIG/USDT', 'SPEC/USDT', 'CARV/USDT', 'TOKEN/USDT', 'CGPT/USDT', 'LUNA/USDT', 'TAI/USDT', 'RSS3/USDT', 'FORT/USDT', 'LMWR/USDT', 'LAI/USDT', 'GTAI/USDT', 'GSWIFT/USDT', 'TRVL/USDT', 'BCUT/USDT', 'NRN/USDT', 'CHRP/USDT', 'RAIN/USDT', 'BBL/USDT', 'GMRX/USDT']


In [3]:
# Fetch ohlcv from Bybit
import numpy as np
import pandas as pd
import datetime as dt
import pytz
import plotly.express as px
import plotly.graph_objects as go

symbols = available_defi_symbols
timeframe = '1d'
limit = 252

# Create an empty list to store DataFrames for each symbol
data_frames = []

for symbol in symbols:
    try:
        ohlcv_data = exchange.fetch_ohlcv(symbol, timeframe, limit=limit)
        df = pd.DataFrame(ohlcv_data, columns=['timestamp', 'open', 'high', 'low', 'close', 'volume'])
        df['symbol'] = symbol

        # Convert timestamp to datetime format for readability
        df['timestamp'] = pd.to_datetime(df['timestamp'], unit='ms')

        data_frames.append(df)
    except Exception as e:
        print(f"Could not fetch data for {symbol}: {e}")

# Concatenate all DataFrames into a single DataFrame
all_data = pd.concat(data_frames, ignore_index=True)
all_data.head()

,timestamp,open,high,low,close,volume,symbol
0,2024-03-10,6.1886,6.3000,5.7510,5.9470,4467334.11,NEAR/USDT
1,2024-03-11,5.9470,7.3200,5.5810,6.6823,9343963.47,NEAR/USDT
2,2024-03-12,6.6823,8.0900,6.6346,8.0733,8284221.42,NEAR/USDT
3,2024-03-13,8.0733,8.5337,7.6011,7.7623,7786205.32,NEAR/USDT
4,2024-03-14,7.7623,8.9146,7.6028,8.8575,11999055.01,NEAR/USDT


In [4]:
# Check min max date
print(all_data['timestamp'].min(), all_data['timestamp'].max())

2024-03-10 00:00:00 2024-11-16 00:00:00


In [6]:
# Filter dublicates and unnecesary timestamps
# Ensure that 'timestamp' is in datetime format
all_data['timestamp'] = pd.to_datetime(all_data['timestamp'])

# Find the latest date in the data
latest_date = all_data['timestamp'].max()

# Calculate the cutoff date for 252 days before the latest date
cutoff_date = latest_date - pd.Timedelta(days=252)

# Filter the data to include only rows within the last 252 days
filtered_data = all_data[all_data['timestamp'] >= cutoff_date]

# Check the new date range to confirm
print("Filtered data range:", filtered_data['timestamp'].min(), "-", filtered_data['timestamp'].max())

# Identify duplicate entries based on 'timestamp' and 'symbol'
duplicates = filtered_data[filtered_data.duplicated(subset=['timestamp', 'symbol'], keep=False)]
print("Duplicate entries:")
duplicates.info()

# Remove duplicate entries, keeping only the first occurrence
filtered_data = filtered_data.drop_duplicates(subset=['timestamp', 'symbol'], keep='first')


Filtered data range: 2024-03-10 00:00:00 - 2024-11-16 00:00:00
Duplicate entries:
<class 'pandas.core.frame.DataFrame'>
Index: 0 entries
Data columns (total 7 columns):
 #   Column     Non-Null Count  Dtype         
---  ------     --------------  -----         
 0   timestamp  0 non-null      datetime64[ns]
 1   open       0 non-null      float64       
 2   high       0 non-null      float64       
 3   low        0 non-null      float64       
 4   close      0 non-null      float64       
 5   volume     0 non-null      float64       
 6   symbol     0 non-null      object        
dtypes: datetime64[ns](1), float64(5), object(1)
memory usage: 0.0+ bytes


In [7]:
# get closed price data
df = filtered_data[['timestamp', 'close', 'symbol']]
df.head()

,timestamp,close,symbol
0,2024-03-10,5.9470,NEAR/USDT
1,2024-03-11,6.6823,NEAR/USDT
2,2024-03-12,8.0733,NEAR/USDT
3,2024-03-13,7.7623,NEAR/USDT
4,2024-03-14,8.8575,NEAR/USDT


In [8]:
# pivot by timestamp and symbol
df_pivoted = df.pivot(index='timestamp', columns='symbol', values='close')

print(df_pivoted.head())
print(df_pivoted.tail())

symbol      AGI/USDT  AIOZ/USDT  ARKM/USDT  BBL/USDT  BCUT/USDT  CARV/USDT  \
timestamp                                                                    
2024-03-10   0.57286     0.7638    3.20786   0.23474    0.30094        NaN   
2024-03-11   0.58374     0.7038    3.06223   0.23186    0.29308        NaN   
2024-03-12   0.60618     0.6872    3.14473   0.20845    0.33999        NaN   
2024-03-13   0.54650     0.7981    3.01128   0.20824    0.44520        NaN   
2024-03-14   0.58526     0.7602    3.21886   0.20226    0.45555        NaN   

symbol      CGPT/USDT  CHRP/USDT  FET/USDT  FORT/USDT  ...  NEAR/USDT  \
timestamp                                              ...              
2024-03-10   0.450675   0.023804    2.7658     0.2509  ...     5.9470   
2024-03-11   0.520328   0.024898    2.7019     0.3156  ...     6.6823   
2024-03-12   0.500831   0.022632    2.6754     0.2892  ...     8.0733   
2024-03-13   0.485226   0.023238    2.6544     0.2688  ...     7.7623   
2024-03-14   0.

In [9]:
# Calculate monthly returns
returns_df = df_pivoted.pct_change().resample("ME").agg(lambda x: (x+1).prod()-1)
# Convert the index of past_cum_return_df to timezone-aware UTC
returns_df.index = returns_df.index.tz_localize('UTC')
returns_df.tail(7)

symbol,AGI/USDT,AIOZ/USDT,ARKM/USDT,BBL/USDT,BCUT/USDT,CARV/USDT,CGPT/USDT,CHRP/USDT,FET/USDT,FORT/USDT,...,NEAR/USDT,NRN/USDT,RAIN/USDT,RENDER/USDT,RSS3/USDT,SPEC/USDT,TAI/USDT,TOKEN/USDT,TRVL/USDT,ZIG/USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-31 00:00:00+00:00,0.246173,0.240603,0.359682,0.042531,-0.123564,0.000000,0.012890,-0.006673,0.056962,0.039916,...,0.170824,0.000000,0.248423,0.000000,-0.085877,1.051827,1.056748,0.403442,-0.169724,0.026327
2024-06-30 00:00:00+00:00,-0.262044,-0.256674,-0.312170,-0.188505,-0.479228,0.000000,-0.237071,-0.475746,-0.330511,-0.286465,...,-0.268194,0.083333,-0.741746,0.000000,-0.311087,-0.518847,0.446516,-0.386592,-0.190083,0.291421
2024-07-31 00:00:00+00:00,-0.247678,0.017777,-0.293644,-0.577514,-0.235030,0.000000,-0.128386,-0.098067,-0.192859,-0.120612,...,-0.058934,-0.379077,-0.140754,0.000000,-0.296483,0.246266,-0.288292,-0.281781,-0.184366,0.065858
2024-08-31 00:00:00+00:00,-0.270114,-0.293225,-0.177279,-0.284291,-0.476288,0.000000,-0.163851,-0.406984,-0.021383,-0.208628,...,-0.192103,-0.323092,0.202535,-0.128631,-0.263879,-0.179538,-0.004909,-0.170667,-0.251468,-0.278775
2024-09-30 00:00:00+00:00,0.407746,0.243884,0.413307,-0.241605,0.425511,0.000000,0.416928,0.119188,0.347311,0.001627,...,0.310809,0.761713,0.166862,0.289116,0.265542,0.285676,0.246927,0.133441,0.412389,0.117441
2024-10-31 00:00:00+00:00,-0.026236,-0.133855,0.105066,-0.475162,-0.288251,0.315214,-0.274536,-0.161103,-0.148588,-0.238830,...,-0.233064,0.002909,-0.135140,-0.258420,-0.186690,0.220705,-0.106895,-0.293617,0.216177,-0.084249
2024-11-30 00:00:00+00:00,0.079340,0.778499,0.258033,0.037037,0.862764,0.166532,-0.005203,0.063149,-0.018200,0.021345,...,0.428307,0.382846,-0.135564,0.426748,-0.026285,0.204117,0.232551,0.439257,0.456845,0.377936


In [10]:
# Create a line plot with Plotly
fig = px.line(returns_df, x=returns_df.index, y=returns_df.columns, title="Monthly returns")
# Adjust the layout for a wider plot
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Returns")
# Show the interactive plot
fig.show()

In [11]:
# rolling cumulative for 6 months return
past_cum_return_df = (returns_df + 1).rolling(6).apply(np.prod) - 1

past_cum_return_df.tail(7)

symbol,1INCH/USDT,A8/USDT,ACA/USDT,AEVO/USDT,ANKR/USDT,ARB/USDT,ATOM/USDT,AURORA/USDT,BAT/USDT,DOT/USDT,...,PTU/USDT,ROSE/USDT,SD/USDT,STRK/USDT,SUSHI/USDT,TON/USDT,WAXP/USDT,XRP/USDT,ZENT/USDT,ZRX/USDT
timestamp,,,,,,,,,,,,,,,,,,,,,
2024-05-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-06-30 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-07-31 00:00:00+00:00,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2024-08-31 00:00:00+00:00,-0.627704,-0.616478,-0.561696,-0.902497,-0.525633,-0.751451,-0.653715,-0.519983,-0.542111,-0.585135,...,-0.552767,-0.683521,-0.521819,-0.853317,-0.721973,0.923787,-0.681915,-0.068903,-0.515160,-0.637148
2024-09-30 00:00:00+00:00,-0.549547,-0.613133,-0.645708,-0.868057,-0.522701,-0.635981,-0.615435,-0.432973,-0.468193,-0.540224,...,-0.494333,-0.440358,-0.562347,-0.798531,-0.575661,0.053136,-0.654491,-0.028135,-0.398015,-0.560100
2024-10-31 00:00:00+00:00,-0.375919,-0.707235,-0.476592,-0.771851,-0.481096,-0.484848,-0.498093,-0.500833,-0.326333,-0.384041,...,-0.359313,-0.278703,-0.544987,-0.697390,-0.282267,-0.061077,-0.509848,0.017572,-0.550165,-0.313742
2024-11-30 00:00:00+00:00,-0.362076,-0.670849,-0.396117,-0.591678,-0.253950,-0.418231,-0.330024,-0.426728,-0.155519,-0.225229,...,-0.251653,-0.181507,-0.091658,-0.606797,-0.320138,-0.119063,-0.381202,1.086360,-0.452302,-0.268565


In [12]:
# Show cumulative return from past 6 months
fig = px.line(past_cum_return_df, x=past_cum_return_df.index, y=past_cum_return_df.columns, title="Rolling cumulative for 6 months return")
fig.update_layout(width=1200, height=600, xaxis_title="Timestamp", yaxis_title="Cumulative Returns")
fig.show()

In [13]:
# Define formation and measurement period
# Define the dates with timezone information
end_of_measurement_period = dt.datetime(2024, 9, 30, tzinfo=pytz.UTC)
formation_period = dt.datetime(2024, 10, 31, tzinfo=pytz.UTC)

end_of_measurement_period_return_df = past_cum_return_df.loc[end_of_measurement_period]
# Transpose the DataFrame so that tickers become rows and returns become a single column
end_of_measurement_period_return_df = end_of_measurement_period_return_df.T
end_of_measurement_period_return_df = end_of_measurement_period_return_df.reset_index()
# Rename columns for clarity
end_of_measurement_period_return_df.columns = ['Symbol', 'Return']
end_of_measurement_period_return_df.head()

,Symbol,Return
0,1INCH/USDT,-0.549547
1,A8/USDT,-0.613133
2,ACA/USDT,-0.645708
3,AEVO/USDT,-0.868057
4,ANKR/USDT,-0.522701


In [14]:
# highest momentum in the positive direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmax()]

Symbol    HNT/USDT
Return    0.264008
Name: 14, dtype: object

In [15]:
# highest momentum in the negative direction
end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df.iloc[:,1].idxmin()]

Symbol    AEVO/USDT
Return    -0.868057
Name: 3, dtype: object

In [16]:
# rank symbols with quantiles
end_of_measurement_period_return_df['rank'] = pd.qcut(end_of_measurement_period_return_df.iloc[:,1], 11, labels=False)
end_of_measurement_period_return_df.head(7)

,Symbol,Return,rank
0,1INCH/USDT,-0.549547,5
1,A8/USDT,-0.613133,3
2,ACA/USDT,-0.645708,2
3,AEVO/USDT,-0.868057,0
4,ANKR/USDT,-0.522701,6
5,ARB/USDT,-0.635981,2
6,ATOM/USDT,-0.615435,3


In [17]:
# Create a bar plot to visualize ranks
fig = px.bar(
    end_of_measurement_period_return_df, 
    x='Symbol', 
    y='Return', 
    color='rank',  # Color by rank to visually distinguish groups
    title="Asset Returns and Ranks",
    labels={'rank': 'Rank'},
)

# Customize layout for readability
fig.update_layout(
    xaxis_title="Symbol",
    yaxis_title="Return",
    coloraxis_colorbar=dict(title="Rank"),
    width=1000,
    height=600
)

fig.show()


In [18]:
# select highest rank
long_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==10,"Symbol"].values
long_stocks

array(['FTT/USDT', 'HNT/USDT', 'TON/USDT', 'XRP/USDT'], dtype=object)

In [19]:
# select lowest rank
short_stocks = end_of_measurement_period_return_df.loc[end_of_measurement_period_return_df["rank"]==0,"Symbol"].values
short_stocks

array(['AEVO/USDT', 'FLIP/USDT', 'OMG/USDT', 'STRK/USDT'], dtype=object)

In [20]:
# long strategy return
from dateutil.relativedelta import relativedelta

long_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(long_stocks)]
long_return_df

symbol
FTT/USDT    0.145378
HNT/USDT   -0.065342
TON/USDT    0.151434
XRP/USDT    1.119113
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [21]:
# short strategy return
short_return_df = returns_df.loc[formation_period + relativedelta(months=1), \
                                   returns_df.columns.isin(short_stocks)]
short_return_df

symbol
AEVO/USDT    0.148670
FLIP/USDT    0.150314
OMG/USDT     0.263404
STRK/USDT    0.247439
Name: 2024-11-30 00:00:00+00:00, dtype: float64

In [23]:
# Define Strategy tester class
class strategyTester:
    """Calculate metrics based on cumulative return"""

    def __init__(self, df: pd.Series, period: str, riskfree_rate: float = 0.03, initial_wealth: int = 1000):
        self.annualized_return_value = self.annualized_return(df, period)
        self.annualized_volatility_value = self.annualized_volatility(df, period)
        self.sharpe_ratio_value = self.sharpe_ratio(df, riskfree_rate)
        self.max_drawdown_value = self.max_drawdown(df, initial_wealth)
        
    # annualized return
    @staticmethod
    def annualized_return(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        annualized_return = (1 + df) ** (factor / len(df)) - 1
        print(f"Annualized return: {annualized_return}")
        return annualized_return

    # annualized volatility
    @staticmethod
    def annualized_volatility(df: pd.Series, period: str):
        if period == 'daily':
            factor = 252
        elif period == 'weekly':
            factor = 52
        elif period == 'monthly':
            factor = 12
        else:
            raise ValueError("Invalid period. Use 'daily', 'weekly', or 'monthly'.")
        
        volatility = df.std() * (factor ** 0.5)
        print(f"Annualized volatility: {volatility}")
        return volatility

    # Sharpe ratio
    @staticmethod
    def sharpe_ratio(df: pd.Series, riskfree_rate: float = 0.03):
        excess_return = df.mean() - riskfree_rate
        volatility = df.std()
        sharpe_ratio = excess_return / volatility if volatility != 0 else np.nan
        print(f"Sharpe ratio: {sharpe_ratio}")
        return sharpe_ratio

    # Drawdown calculation
    @staticmethod
    def drawdown(return_series: pd.Series, initial_wealth: float = 100):
        prior_peaks_series = return_series.cummax()
        drawdown_series = (return_series - prior_peaks_series) / prior_peaks_series
        return pd.DataFrame({
            "Wealth Index": return_series,
            "Prior Peaks": prior_peaks_series,
            "Drawdown": drawdown_series
        })
    
    # Max drawdown
    @staticmethod
    def max_drawdown(df: pd.Series, initial_wealth):
        drawdown_df = strategyTester.drawdown(df, initial_wealth)
        max_drawdown = drawdown_df["Drawdown"].min()
        print(f"Max drawdown: {max_drawdown} with initial wealth: {initial_wealth}")
        return max_drawdown


In [24]:
# long momentum strategy test
long_momentum_strategy_test = strategyTester(long_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
FTT/USDT    0.502613
HNT/USDT   -0.183496
TON/USDT    0.526571
XRP/USDT    8.516174
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 1.8381858323853406
Sharpe ratio: 0.5797653710852276
Max drawdown: -1.4494611915905513 with initial wealth: 1000


In [25]:
# short momentum strategy test
short_momentum_strategy_test = strategyTester(short_return_df, 'monthly', 0.03, 1000)

Annualized return: symbol
AEVO/USDT    0.515604
FLIP/USDT    0.522120
OMG/USDT     1.016634
STRK/USDT    0.941147
Name: 2024-11-30 00:00:00+00:00, dtype: float64
Annualized volatility: 0.21307247108208466
Sharpe ratio: 2.803777388779988
Max drawdown: -0.060609887262734564 with initial wealth: 1000


In [22]:
# Comparison of monthly Long and Short momentum strategies
# Initialize an empty figure
fig = go.Figure()

# Add the first strategy (e.g., long strategy)
fig.add_trace(go.Bar(x=long_return_df.index, y=long_return_df.values, marker_color='blue', name="Long"))

# Add the second strategy (e.g., short strategy)
fig.add_trace(go.Bar(x=short_return_df.index, y=short_return_df.values, marker_color='red', name="Short"))

# Update layout for title and axis labels
fig.update_layout(
    title="Comparison of monthly Long and Short momentum strategies",
    width=1400,
    height=500,
    xaxis_title="Symbol",
    yaxis_title="Cumulative Returns",
    legend_title="Strategies",
)

# Show the plot
fig.show()

In [26]:
# Annualized Return and Max Drawdown from Long and Short Strategies
# Initialize an empty figure
fig = go.Figure()

# Add traces for Long strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Long)", "Max Drawdown (Long)"], 
    y=[long_momentum_strategy_test.annualized_return_value.mean(), long_momentum_strategy_test.max_drawdown_value], 
    marker_color='blue', 
    name="Long"
))

# Add traces for Short strategy
fig.add_trace(go.Bar(
    x=["Annualized Return (Short)", "Max Drawdown (Short)"], 
    y=[short_momentum_strategy_test.annualized_return_value.mean(), short_momentum_strategy_test.max_drawdown_value], 
    marker_color='red', 
    name="Short"
))

# Update layout
fig.update_layout(
    title="Annualized Return and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Values",
    barmode='group',  # Group bars by category
    width=800,
    height=400
)

# Show the plot
fig.show()



In [27]:
# Annualized Volatility and Max Drawdown from Long and Short Strategies
# Create a figure
fig = go.Figure()

# Add bars for the Long Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(long)", "Sharpe Ratio(long)"], 
    y=[long_momentum_strategy_test.annualized_volatility_value, long_momentum_strategy_test.sharpe_ratio_value],
    name="Long",
    marker_color='blue'
))

# Add bars for the Short Strategy - Volatility and Drawdown
fig.add_trace(go.Bar(
    x=["Annualized Volatility(short)", "Sharpe Ratio(short)"], 
    y=[short_momentum_strategy_test.annualized_volatility_value, short_momentum_strategy_test.sharpe_ratio_value],
    name="Short",
    marker_color='red'
))

# Update layout for grouped bars
fig.update_layout(
    title="Annualized Volatility and Max Drawdown from Long and Short Strategies",
    xaxis_title="Metrics",
    yaxis_title="Metric Value",
    barmode='group',  # Group bars side-by-side
    width=800,
    height=400
)

# Show the plot
fig.show()
